In [19]:
import os
import mido
import time

TICKS_PER_BEAT = 220

In [20]:
in_file = "../data/datasets/test/beats-060-06_threequarters_t00s00.mid"
midi = mido.MidiFile(in_file)
midi.print_tracks()

=== Track 0
MetaMessage('track_name', name='beats-060-06_threequarters', time=0)
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=36, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=36, notated_32nd_notes_per_beat=8, time=0)
Message('note_on', channel=0, note=70, velocity=111, time=220)
Message('note_off', channel=0, note=70, velocity=64, time=41)
Message('note_on', channel=0, note=70, velocity=17, time=13)
Message('note_off', channel=0, note=70, velocity=64, time=41)
Message('note_on', channel=0, note=70, velocity=111, time=13)
Message('note_off', channel=0, note=70, velocity=64, time=41)
Message('note_on', channel=0, note=70, velocity=18, time=13)
Message('note_off', channel=0, note=70, velocity=64, time=41)
Message('note_on', channel=0, note=70, velocity=33, time=13)
Message('note_off', channel=0, note=70, velocity=64, time=41)
Message('note_on', channel=0, note=70, velocity=59, time=13)
Mess

In [21]:
def get_messages_by_beat(
    midi_path: str, tempo: int
) -> list[list[mido.Message], list[mido.MetaMessage]]:
    midi = mido.MidiFile(midi_path)

    # calculate ticks per second and seconds per beat
    ticks_per_second = TICKS_PER_BEAT * tempo / 60
    seconds_per_beat = 60 / tempo

    # convert all messages to absolute timing
    absolute_msgs = []
    current_tick = 0
    for msg in midi.tracks[0]:
        current_tick += msg.time
        absolute_msgs.append((current_tick, msg.copy()))

    if not absolute_msgs:
        console.log(f"[red]no messages found in {midi_path}[/red]")
        return []

    # find the last tick to determine number of beats
    last_tick = absolute_msgs[-1][0]
    num_beats = int(last_tick / TICKS_PER_BEAT) + 1

    # initialize list of beats
    beats = [
        [
            [],
            [
                mido.MetaMessage("text", text=f"tick {i}", time=0),
                mido.MetaMessage("text", text=f"tick {i+1}", time=TICKS_PER_BEAT),
            ],
        ]
        for i in range(num_beats)
    ]

    # track active notes and their start times
    active_notes = {}  # (channel, note) -> (start_tick, start_beat)

    for tick, msg in absolute_msgs:
        # calculate which beat this message belongs to
        beat_idx = int(tick / TICKS_PER_BEAT)

        if msg.type == "note_on" and msg.velocity > 0:
            # note on event - store start time and beat
            active_notes[(msg.channel, msg.note)] = (tick, beat_idx)
            beats[beat_idx][0].append(msg.copy())
        elif msg.type == "note_off" or (msg.type == "note_on" and msg.velocity == 0):
            # note off event - find where the note started
            key = (msg.channel, msg.note)
            if key in active_notes:
                start_tick, start_beat = active_notes[key]
                # add note off to the same beat as note on
                beats[start_beat][0].append(msg.copy())
                del active_notes[key]
        else:
            # non-note message - add to current beat
            beats[beat_idx][0].append(msg.copy())

    for beat_msgs in beats:
        if not beat_msgs:
            continue

        # make a copy and sort by absolute time
        sorted_msgs = sorted(beat_msgs[0], key=lambda msg: msg.time)

        # convert to relative timing by working backwards
        prev_time = sorted_msgs[-1].time if sorted_msgs else 0
        for i in range(len(sorted_msgs) - 1, 0, -1):
            current_time = sorted_msgs[i].time
            sorted_msgs[i].time = current_time - sorted_msgs[i - 1].time
        if sorted_msgs:
            sorted_msgs[0].time = sorted_msgs[0].time

        # update the beat messages with the new relative timing
        beat_msgs[0][:] = sorted_msgs

    return beats

In [24]:
bb = get_messages_by_beat(in_file, 60)
bb

[[MetaMessage('track_name', name='beats-060-06_threequarters', time=0),
  MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=36, notated_32nd_notes_per_beat=8, time=0),
  MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=36, notated_32nd_notes_per_beat=8, time=0)],
 [Message('note_on', channel=0, note=70, velocity=111, time=0),
  Message('note_off', channel=0, note=70, velocity=64, time=41),
  Message('note_on', channel=0, note=70, velocity=17, time=13),
  Message('note_off', channel=0, note=70, velocity=64, time=41),
  Message('note_on', channel=0, note=70, velocity=111, time=13),
  Message('note_off', channel=0, note=70, velocity=64, time=41),
  Message('note_on', channel=0, note=70, velocity=18, time=13),
  Message('note_off', channel=0, note=70, velocity=64, time=41),
  Message('note_on', channel=0, note=70, velocity=33, time=13),
  Message('note_off', channel=0, note=70, velocity=64, time=41)],
 [Message('note_on', channel=0, note=7